## Run simulation and optimizations for the folded cascode example


In [1]:
import json
import circuits as cir
import optimizers as opt
import ngspice
import pandas as pd

import importlib
importlib.reload(cir)
importlib.reload(opt)
importlib.reload(ngspice)


<module 'ngspice' from '/home/nlourenco/vscode_aida/BAIcO/src/ngspice.py'>

In [2]:
#load the cicruit definitions and targets defined in circuit_setup.json
folded_cascode = cir.Circuit("./circuit_examples/ptm130_folded_cascode/", corners=None)

#load a sample sizing
with open("./circuit_examples/ptm130_folded_cascode/sizing_example.json", 'r') as file:
    sizing = json.load(file)

simulation_result = folded_cascode.simulate(sizing)


print("Simulation Results Typ Only")

df = pd.DataFrame(simulation_result[0]).transpose()

df['CL'] *= 1e12  
df.rename(columns = {"CL": "CL[pF]"}, inplace=True)  
print(df)


Simulation Results Typ Only
    CL[pF]  DELTA_MNM1  DELTA_MNM2  DELTA_MNM3  DELTA_MNM4  DELTA_MNM5  \
TT     6.0    0.585495    0.739938    0.617564    0.619046    0.413995   

    DELTA_MNM6  DELTA_MNM7  DELTA_MNM8  DELTA_MPM0  ...  VOV_MNM5  VOV_MNM6  \
TT    0.639173    0.402403    0.402872    0.472546  ...  0.001432  0.000963   

    VOV_MNM7  VOV_MNM8  VOV_MPM0  VOV_MPM1  VOV_MPM3  VOV_MPM4  VOV_MPM5  \
TT -0.016224 -0.016224   0.37476  0.103984  1.307041  1.307041  0.405695   

    VOV_MPM6  
TT  0.407171  

[1 rows x 42 columns]


In [3]:
#load the cicruit definitions and targets defined in circuit_setup.json with corners from corners.inc
folded_cascode = cir.Circuit("./circuit_examples/ptm130_folded_cascode/")

#load a sample sizing
with open("./circuit_examples/ptm130_folded_cascode/sizing_example.json", 'r') as file:
    sizing = json.load(file)

simulation_result = folded_cascode.simulate(sizing)


print("Simulation Results All Corners")
df = pd.DataFrame(simulation_result[0]).transpose()

# rescale CL to show not 0 in df
df['CL'] *= 1e12
df.rename(columns = {"CL": "CL[pF]"}, inplace=True)  
print(df)

Simulation Results All Corners
                       CL[pF]       GDC         GBW  PM_NEGATIVE        GPS  \
SS_VDD_MIN_TEMP_MIN       6.0  80.83673  39554440.0    -114.2310  -5.460861   
TT                        6.0  88.39221  31793610.0    -122.3191  -3.418371   
SS_VDD_MAX_TEMP_MIN       6.0  69.53510  46737210.0    -108.8653 -14.417560   
SS_VDD_MIN_TEMP_MAX       6.0 -86.95084         NaN          NaN -75.768650   
FF_VDD_MAX_TEMP_MIN       6.0  68.64928  58458860.0    -107.7896 -14.566390   
FF_VDD_MAX_TEMP_MAX       6.0  87.51435  32338310.0    -122.7734 -18.703350   
FF_VDD_MIN_TEMP_MAX       6.0  98.37739  18233250.0    -132.4969   6.757086   
SS_VDD_MAX_TEMP_MAX       6.0  95.32268  23641690.0    -131.6557  -1.444409   
FF_VDD_MIN_TEMP_MIN       6.0  72.23919  42161240.0    -106.8764 -21.388510   
SNFP_VDD_MIN_TEMP_MIN     6.0  68.89267  40677850.0    -108.4439 -18.217450   
SNFP_VDD_MIN_TEMP_MAX     6.0 -87.04048         NaN          NaN -63.453340   
SNFP_VDD_MAX_TEMP_MIN

In [4]:
obj,cstr,log = folded_cascode.target.evaluate(simulation_result[0])

print("Objectives")

print( {folded_cascode.objectives[i][0]:(obj[i]*folded_cascode.objectives[i][1]) for i in range(len(obj))})

Objectives
{'IDD': 0.001285138, 'GBW': -inf, 'GDC': -98.38497}


In [9]:
  
nsga2 = opt.NSGA2()

for pop, pop_obj,  pop_cstr, pop_data, evals, front_no in nsga2.minimize(
    folded_cascode, pop_size=256, evaluations=256*500, mutation=0.15):
    print(evals)
    print(pop_obj[pop_cstr.argmax()] , pop_cstr[pop_cstr.argmax()])
    print(pop_data[pop_cstr.argmax()][1]['TT'])



128000
[ 2.945921e-03 -1.407357e+07 -4.492332e+01] [-99.93365329]
{'IDD_lt': (0.00035, 0.0008358312, -1.3880891428571427), 'FOM_gt': (500.0, 137.79586117388297, -0.7244082776522341), 'VOV_MNM7_gt': (0.05, 0.02203658, -0.5592684), 'VOV_MNM8_gt': (0.05, 0.02204395, -0.559121), 'DELTA_MPM5_gt': (0.1, 0.09969517, -0.0030483000000000593), 'DELTA_MPM6_gt': (0.1, -0.00703182, -1.0703182)}


KeyboardInterrupt: 

In [6]:
import random
from datetime import datetime
import numpy as np
import json
import optimizers as opt
import circuits as cir
import ngspice
import pandas as pd

import importlib
importlib.reload(opt)
importlib.reload(cir)
importlib.reload(ngspice)

seed = 42
np.random.seed(seed)
random.seed(seed)
    
nsga2 = opt.NSGA2()
i = 0
folded_cascode = cir.Circuit("./circuit_examples/ptm130_folded_cascode/", corners=None)
for pop, pop_obj,  pop_cstr, pop_data, evals, front_no in nsga2.minimize(
        folded_cascode,pop_size=256, evaluations=256*100, mutation=0.1, crossover=0.6):
        
        print(i, pop_cstr[pop_cstr.argmax()], datetime.now().time())
        with open("./run/ptm130_folded_cascode/history256_{}_{}.json".format(seed, i), "w") as file: 
            json.dump(
            {
                "pop":json.loads(pd.DataFrame(data=pop, columns=folded_cascode.parameters).to_json(orient="records")),
                "obj":json.loads(pd.DataFrame(data=pop_obj, columns=folded_cascode.objectives).to_json(orient="records")),
                "cstr":pop_cstr.tolist(),
                "data":pop_data,
                "evals":evals,
                "fn":[ str(fn) for fn in front_no.tolist()]
            },
            file  
            )
        i = i + 1


0 [-2.46806649] 14:04:21.579429


AttributeError: module 'circuits' has no attribute 'parameters'

In [ ]:
print(history[0][5])

